# Example queries for biological entities on COVID-19-Net Knowledge Graph
[Work in progress]

This notebook demonstrates how to run Cypher queries to retrieve data about biological entities in the Knowledge Graph.

In [1]:
import datetime
import pandas as pd
from py2neo import Graph

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

#### Connect to COVID-19-Net Knowledge Graph

In [3]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### What kind of entities, including bioentities, does the KG contain?

In [4]:
query = """
MATCH (n:NodeMetadata)
RETURN n.name, n.shortDescription, n.description, n.example, n.definitionSource, n.dataProviders
"""
graph.run(query).to_data_frame()

,n.name,n.shortDescription,n.description,n.example,n.definitionSource,n.dataProviders
0,Location,Geographic location,A geographic location,"World, ..., Country, State, Country, City, Cru...",None,"[GeoNames, UNSD, USCensus, HUD, JHU]"
1,World,The World,Top level location,None,None,None
2,UNRegion,Continental regions,Continental regions according to the M49 stan...,Americas,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
3,UNSubRegion,Subcontinental regions,Subcontinental regions according to the M49 st...,Latin America and the Caribbean,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
4,UNIntermediateRegion,Subdivisions of subcontinental regions,Subdivisions of subcontinental regions accordi...,Caribbean,https://unstats.un.org/unsd/methodology/m49/,[UNSD]
5,Country,Countries and dependent Territories,Countries and dependent Territories defined b...,United States,http://www.geonames.org/,[GeoNames]
6,Admin1,"State, Province, Municipality","First administrative divisions, e.g, State, Pr...",California,http://www.geonames.org/,"[GeoNames, USCensus]"
7,Admin2,County,Second administrative divisions: County in the US,San Diego County,http://www.geonames.org/,"[GeoNames, USCensus]"
8,City,City,City,San Diego,http://www.geonames.org/,"[GeoNames, USCensus]"
9,PostalCode,Postal Code,"E.g., a ZIP Code is a postal code used by the ...",92121,http://purl.obolibrary.org/obo/OPMI_0000120,"[GeoNames, HUD]"


### Fuzzy full-text search for bioentities
Results are ordered by match score. 

Note, proteins may have one or more protein names. Proteins that have been cleaved have the fullLength flag set to `False`.

In [5]:
query = """
CALL db.index.fulltext.queryNodes("bioentities", "Spike") YIELD node, score
RETURN node.name AS name, node.fullLength AS fullLength, labels(node), score
"""
df = graph.run(query).to_data_frame()
df.head(25)

,name,fullLength,labels(node),score
0,Spike glycoprotein,True,[Protein],5.124448
1,Spike glycoprotein,False,[Protein],5.124448
2,Spike glycoprotein,None,[ProteinName],5.124448
3,Spike glycoprotein,None,[ProteinName],5.124448
4,Spike protein S1,False,[Protein],4.623253
5,Spike protein S2,False,[Protein],4.623253
6,Spike protein S2',False,[Protein],4.623253
7,Spike protein S1,None,[ProteinName],4.623253
8,Spike protein S2,None,[ProteinName],4.623253
9,Spike protein S2',None,[ProteinName],4.623253


### Exact full-text search for bioentities
For exact matches, enclose the phrase with `\"`.

In [6]:
query = """
CALL db.index.fulltext.queryNodes("bioentities", '\"Spike glycoprotein\"') YIELD node
RETURN node.name AS name, labels(node), node.accession AS accession, node.fullLength AS fullLength
"""
df = graph.run(query).to_data_frame()
df.head(25)

,name,labels(node),accession,fullLength
0,Spike glycoprotein,[Protein],uniprot:P0DTC2,True
1,Spike glycoprotein,[Protein],uniprot:P0DTC2,False
2,Spike glycoprotein,[ProteinName],uniprot:P0DTC2,None
3,Spike glycoprotein,[ProteinName],uniprot:P0DTC2,None


### Which organisms have been identified as hosts for SARS-CoV-2

In [7]:
query = """
MATCH (h:Host)-[:CARRIES]->(:Strain)<-[:HAS_STRAIN]-(p:Pathogen{name: 'SARS-CoV-2'})
RETURN distinct(h.name) AS name, h.scientificName AS scientificName, h.id AS taxonomy
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy
0,human,Homo sapiens,taxonomy:9606
1,house mouse,Mus musculus,taxonomy:10090
2,European mink,Mustela lutreola,taxonomy:9666
3,carnivores,Canidae,taxonomy:9608
4,Malayan tiger,Panthera tigris jacksoni,taxonomy:419130
5,Malayan horseshoe bat,Rhinolophus malayanus,taxonomy:608659
6,Malayan pangolin,Manis javanica,taxonomy:9974
7,cat,Felis catus,taxonomy:9685


### Which Pathogens have caused Coronavirus Outbreaks?

In [8]:
query = """
MATCH (p:Pathogen)-[:CAUSES]->(o:Outbreak)
RETURN p.name AS name, p.scientificName AS scientificName, p.id AS taxonomy, o.id AS outbreak, o.startDate AS startDate
"""
graph.run(query).to_data_frame()

,name,scientificName,taxonomy,outbreak,startDate
0,SARS-CoV-2,Severe acute respiratory syndrome coronavirus 2,taxonomy:2697049,COVID-19,2019
1,MERS-CoV,Middle East respiratory syndrome-related coron...,taxonomy:1335626,MERS,2012
2,SARS-CoV,Severe acute respiratory syndrome-related coro...,taxonomy:694009,SARS,2003


### Which PubMedCentral articles mention strain datasets about different hosts?

In [9]:
query = """
MATCH (p:Publication)-[:MENTIONS]->(s:Strain)<-[:CARRIES]-(h:Host)
RETURN p.id AS pmc, s.name AS name, s.collectionDate  AS collectionDate, h.name AS host, h.id AS host_id
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame().head(20)

,pmc,name,collectionDate,host,host_id
0,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P4L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
1,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P1E/2017,2017-01-01,Malayan pangolin,taxonomy:9974
2,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P5L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
3,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P2V/2017,2017-01-01,Malayan pangolin,taxonomy:9974
4,pmc:PMC7166309,BetaCoV/pangolin/Guangxi/P5E/2017,2017-01-01,Malayan pangolin,taxonomy:9974
5,pmc:PMC7228214,BetaCoV/pangolin/Guangxi/P4L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
6,pmc:PMC7211627,BetaCoV/pangolin/Guangxi/P5L/2017,2017-01-01,Malayan pangolin,taxonomy:9974
7,pmc:PMC7166309,BetaCoV/pangolin/Guandong/1/2020,2019-01-01,Malayan pangolin,taxonomy:9974
8,pmc:PMC7228214,BetaCoV/pangolin/Guandong/1/2020,2019-01-01,Malayan pangolin,taxonomy:9974
9,pmc:PMC7205519,BetaCoV/pangolin/Guandong/1/2020,2019-01-01,Malayan pangolin,taxonomy:9974


### What are the genes and gene products of the SARS-CoV-2 Virus?
This query lists the genes and proteins encoded by the SARS-CoV-2 NC_045512 reference genome. MN908947 represents the original un-annotated dataset submitted to NCBI. This is the first sequenced genome of SARS-CoV-2 collected in Wuhan.

In [10]:
query = """
MATCH (s:Strain)-[:HAS]->(g:Gene)-[:ENCODES]->(p:Protein)
RETURN s.id as referenceGenome, s.name as name, s.collectionDate  as collectionDate, 
       g.name as gene, g.id as geneId, p.name as protein, p.fullLength as fullLength, p.id as protein_id 
ORDER by s.collectionDate
"""
graph.run(query).to_data_frame()

,referenceGenome,name,collectionDate,gene,geneId,protein,fullLength,protein_id
0,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,ORF1ab,ncbiprotein:NC_045512-266-21555,Replicase polyprotein 1ab,True,md5:e6608b50fcd6e004708a875615ddf2d9
1,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,ORF1ab,ncbiprotein:NC_045512-266-13483,Replicase polyprotein 1a,True,md5:e781b58591b8dbdd15f84dcbdec82105
2,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,S,ncbiprotein:NC_045512-21563-25384,Spike glycoprotein,True,md5:4c35f09aac2f7be4f3cffd30c6aecac8
3,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,ORF3a,ncbiprotein:NC_045512-25393-26220,Protein 3a,True,md5:f5c8b89ceac3f14e456577557df1ef40
4,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,E,ncbiprotein:NC_045512-26245-26472,Envelope small membrane protein,True,md5:375e0f905c315e06a99c80b736c125d2
5,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,M,ncbiprotein:NC_045512-26523-27191,Membrane protein,True,md5:1cd6abff79ad3633e17582eb0e576539
6,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,ORF6,ncbiprotein:NC_045512-27202-27387,Non-structural protein 6,True,md5:90b50e0be9abd893bd32b163d6933f0c
7,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,ORF7a,ncbiprotein:NC_045512-27394-27759,Protein 7a,True,md5:f65213344e2e68de1cae4feb9c5e07b1
8,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,ORF7b,ncbiprotein:NC_045512-27756-27887,Protein non-structural 7b,True,md5:c7f0179da4ca26456ee905081f485cc8
9,ncbiprotein:NC_045512,Wuhan-Hu-1,2019-12-24,ORF8,ncbiprotein:NC_045512-27894-28259,Non-structural protein 8,True,md5:3a77ad9207d29beacfbf6f18b0da9e26


### How many complete SARS-CoV-2 genomes have been sequenced in each country?
Here we aggregate the number of genomes over up to 3 hops to the UN Region level:

`City -> Admin2(county) -> Admin1(state, province) -> Country`

using the variable-length relationship `[:IN*0..3]`.

This number includes only complete high-quality genomes as determined by the [China National Center for Bioinformation, 2019 Novel Coronavirus Resource (2019nCoVR)](https://bigd.big.ac.cn/ncov/release_genome).

Note, some strains have been deposited under different names in multiple repositories. Therefore, the numbers below includes some duplicates.

In [11]:
query = """
MATCH (s:Strain)-[:FOUND_IN]->(l:Location)-[:IN*0..3]->(c:Country)
RETURN count(s) AS count, c.name AS country
ORDER by count DESC
"""
graph.run(query).to_data_frame()

,count,country
0,16115,United States
1,5784,United Kingdom
2,1257,China
3,1189,Australia
4,1008,India
5,907,Netherlands
6,635,Canada
7,415,Portugal
8,332,Denmark
9,316,Singapore


### How many complete SARS-CoV-2 genome have been sequenced in each UN Region?
Here we aggregate the number of genomes over up to 6 hops to the UN Region level:

`City -> Admin2(county) -> Admin1(state, province) -> Country -> UNSubRegion -> UNIntermediateRegion -> UNRegion`

using the variable-length relationship `[:IN*0..6]`.

In [12]:
query = """
MATCH (s:Strain)-[:FOUND_IN]->(l:Location)-[:IN*0..6]->(u:UNRegion)
RETURN count(s) AS count, u.name
ORDER by count DESC
"""
graph.run(query).to_data_frame()

,count,u.name
0,19960,Americas
1,9099,Europe
2,3684,Asia
3,1285,Oceania
4,180,Africa
